In [ ]:
import numpy as np
import os
import plotly.graph_objects as go

from chimera_fgo.util.kitti import process_kitti_gt, load_icp_results
from chimera_fgo.util.plot import plot_trajectories, plot_MC

%load_ext autoreload
%autoreload 2

In [ ]:
import plotly.express as px
colors = px.colors.qualitative.D3

In [ ]:
kitti_seq = '0028'
fgo_run = 'fgo_200m_800start_15runs_100w'
fgo_blind_run = 'fgo_200m_8runs_100w_blind'
N = 1900
N_BLIND = 1900

In [ ]:
fgo_results_path = os.path.join(os.getcwd(), '..', '..', 'results', kitti_seq, '2023-01-30', fgo_run)
fgo_results_files = os.listdir(fgo_results_path)

fgo_blind_results_path = os.path.join(os.getcwd(), '..', '..', 'results', kitti_seq, 'final', fgo_blind_run)
fgo_blind_results_files = os.listdir(fgo_blind_results_path)

start_idx = 1550 if kitti_seq == '0028' else 0
gtpath = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)
data_path = os.path.join(os.getcwd(), '..', '..', 'data', 'kitti', kitti_seq, 'icp')
lidar_Rs, lidar_ts, lidar_positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

n_runs = min(len(fgo_results_files), len(fgo_blind_results_files))
fgo_blind_err_avg = np.zeros(N_BLIND)
fgo_err_avg = np.zeros(N)
time = np.arange(N) / 10

fig = go.Figure()
fig.add_trace(go.Scatter(x=[100, 100], y=[0, 300], mode='lines', name='Start of attack', line=dict(color='red', dash='dash'), showlegend=False))

for i in range(n_runs):
    fgo_results = np.load(os.path.join(fgo_results_path, fgo_results_files[i]))
    fgo_blind_results = np.load(os.path.join(fgo_blind_results_path, fgo_blind_results_files[i]))
    fgo_positions = fgo_results['fgo_positions']
    fgo_blind_positions = fgo_blind_results['positions']
    
    fgo_blind_err = np.linalg.norm(fgo_blind_positions - gt_enu[:N_BLIND], axis=1)
    fgo_err = np.linalg.norm(fgo_positions[:N] - gt_enu[:N], axis=1)
    # fgo_blind_err_avg += fgo_blind_err
    # fgo_err_avg += fgo_err
    showlegend = True if i == 0 else False
    fig.add_trace(go.Scatter(x=time, y=fgo_blind_err, name='Naive FGO', line=dict(color='orange'), showlegend=showlegend))
    fig.add_trace(go.Scatter(x=time, y=fgo_err, name='SR FGO', line=dict(color='blue'), showlegend=showlegend))


lidar_err = np.linalg.norm(lidar_positions[:N] - gt_enu[:N], axis=1)
fgo_blind_err_avg /= n_runs
fgo_err_avg /= n_runs

fig.add_trace(go.Scatter(x=time, y=lidar_err, name='Odometry only', line=dict(color='green')))
# fig.add_trace(go.Scatter(x=np.arange(len(fgo_blind_err_avg)), y=fgo_blind_err_avg, name='FGO blind'))
# fig.add_trace(go.Scatter(x=np.arange(len(fgo_err_avg)), y=fgo_err_avg, name='FGO'))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='L2 norm error [m]')
fig.update_layout(legend=dict(x=0.05, y=0.98), font=dict(size=18))

In [ ]:
np.mean(fgo_blind_err[:1000])

In [ ]:
np.mean(fgo_err[:1000])